In [7]:
from pathlib import Path
import subprocess
import numpy as np
from PIL import Image

# Create output directory if it doesn't exist
Path("./svg").mkdir(exist_ok=True)

def image_to_svg(input_path, output_path):
    """Convert a PNG image to SVG using potrace with optimized parameters for clean lines."""
    # Convert to black and white image first
    img = Image.open(input_path).convert('L')
    # Apply strong threshold for crisp edges
    img = img.point(lambda x: 0 if x < 200 else 255, '1')
    
    # Save temporary PBM file (potrace input format)
    temp_pbm = str(Path(input_path).with_suffix('.pbm'))
    img.save(temp_pbm)
    
    try:
        # Run potrace with optimized parameters for clean lines
        subprocess.run([
            'potrace',
            '--svg',                    # Output SVG format
            '--alphamax', '0.1',        # Very low threshold for corners (makes sharper corners)
            '--turdsize', '4',          # Remove small artifacts
            '--turnpolicy', 'black',    # Optimize for black regions
            '--opttolerance', '0.1',    # Very precise curve optimization
            '--unit', '1000',           # High resolution output
            '--scale', '1.0',           # Maintain scale
            '--blacklevel', '0.5',      # Balanced black/white threshold
            '--output', str(output_path),
            temp_pbm
        ], check=True)
    finally:
        # Clean up temporary file
        Path(temp_pbm).unlink(missing_ok=True)

# Process all PNG files too
for png_file in Path("./png").glob("*.png"):
    svg_file = Path("./svg") / f"{png_file.stem}.svg"
    print(f"Converting {png_file} to {svg_file}")
    
    # Load PNG as numpy array
    img = np.array(Image.open(png_file).convert('L')) / 255.0
    array_to_svg(img, svg_file)

print("Conversion complete!")

Converting png/2.png to svg/2.svg
Converting png/3.png to svg/3.svg
Converting png/1.png to svg/1.svg
Conversion complete!
